In [34]:
from datetime import datetime
import calendar
import pandas as pd
import matplotlib.pyplot as plt
from riak import RiakClient

client = RiakClient()

table = "WaterMeterMonthlyRollUp"

In [35]:
def convert_to_epoch ( date_to_convert ):
    return calendar.timegm(datetime.timetuple( date_to_convert )) * 1000

In [36]:
def ts_obj_to_list_of_lists (ts_obj):
    list_to_return = []
    for row in ts_obj.rows:
        list = []
        for i in range(len(row)):
            list.append(row[i])
        list_to_return.append(list)
    return list_to_return

In [37]:
start_date = datetime(2016, 1, 1, 0, 00)
end_date = datetime(2017, 2, 1, 0, 00)

query = """\
    SELECT *
    FROM WaterMeterMonthlyRollUp
    WHERE reading_date >= {} AND reading_date < {} AND
        customer_id = 'CUSTOMER-0001' AND meter_id = 'METER-0001'
""".format( convert_to_epoch( start_date ), convert_to_epoch( end_date ) )

data_set = client.ts_query(table, query)

for row in data_set.rows:
    print row

['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 2, 1, 0, 0), 2L, 2016L, 3756.0, 225.35999999999999]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 3, 1, 0, 0), 3L, 2016L, 3552.5, 213.15]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 4, 1, 0, 0), 4L, 2016L, 3798.5, 227.91]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 5, 1, 0, 0), 5L, 2016L, 3657.0, 219.42]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 6, 1, 0, 0), 6L, 2016L, 3758.5, 225.51]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 7, 1, 0, 0), 7L, 2016L, 4995.5, 299.72999999999996]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 8, 1, 0, 0), 8L, 2016L, 5762.5, 345.75]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 9, 1, 0, 0), 9L, 2016L, 6128.5, 367.71]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 10, 1, 0, 0), 10L, 2016L, 3675.5, 220.53]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 11, 1, 0, 0), 11L, 2016L, 3780.0, 226.79999999999998]


In [38]:
boring_list = ts_obj_to_list_of_lists( data_set )
df = pd.DataFrame( boring_list )
df.columns = ['Customer','Meter','Date','Month','Year', 'Gallons', 'Bill']
df.set_index('Date', False, False, True, True)

In [44]:
print df
print '\n'
print df.describe()

                 Customer       Meter       Date  Month  Year  Gallons    Bill
Date                                                                          
2016-02-01  CUSTOMER-0001  METER-0001 2016-02-01      2  2016   3756.0  225.36
2016-03-01  CUSTOMER-0001  METER-0001 2016-03-01      3  2016   3552.5  213.15
2016-04-01  CUSTOMER-0001  METER-0001 2016-04-01      4  2016   3798.5  227.91
2016-05-01  CUSTOMER-0001  METER-0001 2016-05-01      5  2016   3657.0  219.42
2016-06-01  CUSTOMER-0001  METER-0001 2016-06-01      6  2016   3758.5  225.51
2016-07-01  CUSTOMER-0001  METER-0001 2016-07-01      7  2016   4995.5  299.73
2016-08-01  CUSTOMER-0001  METER-0001 2016-08-01      8  2016   5762.5  345.75
2016-09-01  CUSTOMER-0001  METER-0001 2016-09-01      9  2016   6128.5  367.71
2016-10-01  CUSTOMER-0001  METER-0001 2016-10-01     10  2016   3675.5  220.53
2016-11-01  CUSTOMER-0001  METER-0001 2016-11-01     11  2016   3780.0  226.80
2016-12-01  CUSTOMER-0001  METER-0001 2016-12-01    

In [40]:
gallons_only = []
for i in range( len( boring_list ) ):
    new_row = [ boring_list[i][2], boring_list[i][5] ]
    gallons_only.append( new_row )

df2 = pd.DataFrame( gallons_only )
df2.columns = ['Date', 'Gallons']
df2.set_index('Date', False, False, True, True)

print df2

                 Date  Gallons
Date                          
2016-02-01 2016-02-01   3756.0
2016-03-01 2016-03-01   3552.5
2016-04-01 2016-04-01   3798.5
2016-05-01 2016-05-01   3657.0
2016-06-01 2016-06-01   3758.5
2016-07-01 2016-07-01   4995.5
2016-08-01 2016-08-01   5762.5
2016-09-01 2016-09-01   6128.5
2016-10-01 2016-10-01   3675.5
2016-11-01 2016-11-01   3780.0
2016-12-01 2016-12-01   3634.0
2017-01-01 2017-01-01   3798.5


In [42]:
df2.plot()